Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23102-23128-adrenal-altius-mouse-cross.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [6]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award
0,f5b7565e-64b7-4a97-864e-2ae4c33784c8,ENCBS886OBH,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND4_Male 03_ENC4_cDNA438,23102,ENC4_cDNA438,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
1,NaN,ENCBS623ZPY,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND4_Male 04_ENC4_cDNA439,23103,ENC4_cDNA439,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
2,NaN,ENCBS886BLQ,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND4_Female 03_ENC4_cD...,23104,ENC4_cDNA440,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
3,NaN,ENCBS118MVG,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND4_Female 04_ENC4_cD...,23105,ENC4_cDNA441,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
4,NaN,ENCBS588OTO,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND10_Male 03_ENC4_cDN...,23106,ENC4_cDNA442,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
5,NaN,ENCBS368WXW,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND10_Male 04_ENC4_cDN...,23107,ENC4_cDNA443,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
6,NaN,ENCBS824HLN,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND10_Female 03_ENC4_c...,23108,ENC4_cDNA444,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
7,NaN,ENCBS622FYD,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND10_Female 04_ENC4_c...,23109,ENC4_cDNA445,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
8,NaN,ENCBS479RMT,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND14_3x Males 03_ENC4...,23110,ENC4_cDNA446,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443
9,NaN,ENCBS361GTM,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,adrenal gland_B6Cast F1 PND14_3x Males 04_ENC4...,23111,ENC4_cDNA447,/organisms/mouse,/sources/jackson-labs/,barbara-wold,UM1HG009443


# Register Biosamples

In [7]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True,
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [8]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [9]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23102-23128-adrenal-altius-mouse-cross.xlsx
0


In [10]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [11]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [12]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [13]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True,
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23102-23128-adrenal-altius-mouse-cross.xlsx
0


In [14]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [17]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

54
